Can be used on every other website which is using CKAN as well, e.g. the European Data Portal.

http://suche.transparenz.hamburg.de/

https://www.europeandataportal.eu/data/en/ (doesn't work since they are using an old version and size='None' 20170217 tma)

https://data.gov.uk/data/search

<h2>First approach</h2>

This job loads all resources from the transparenz.hamburg.de website, translates it into a dataframe and shows the biggest files on the website (not possible to do this over the web page directly)


In [2]:
## Example from http://docs.ckan.org/en/ckan-2.2.3/api.html#making-an-api-request
## transparenz.hamburg.de is based on CKAN

import json
import pandas as pd
from pandas import DataFrame
from pandas.io.json import json_normalize

#import urllib2
#py 3 http://stackoverflow.com/questions/2792650/python3-error-import-error-no-module-name-urllib2
from urllib.request import urlopen
#import urllib
#py 3 http://stackoverflow.com/questions/31827012/python-importing-urllib-quote
import urllib.parse
import json
import pprint


In [10]:
dfj = DataFrame()

    ##
    ## When limit is disabled, won't work.With offset and paging it does, putting things in smaller bits
    ##

my_offset = 0
offset_step = 250
still_has_data = True

while still_has_data:
    print('Offset: ' + str(my_offset))
    # Use the json module to dump a dictionary to a string for posting.
    data_string = urllib.parse.quote(json.dumps({'id': 'data-explorer', 'limit': offset_step, 'offset': my_offset}))

    #py 3 http://stackoverflow.com/questions/13175253/python-error-type-error-post-data-should-be-bytes-also-user-agent-issue
    data_string = data_string.encode('ascii')

    # Make the HTTP request.
    #response = urllib2.urlopen('http://suche.transparenz.hamburg.de/api/action/group_list',
    #        data_string)
    # py 3
    response = urlopen('http://suche.transparenz.hamburg.de/api/action/current_package_list_with_resources', data_string)
    assert response.code == 200

    # Use the json module to load CKAN's response into a dictionary.
    response_dict = json.loads(response.read())

    # Check the contents of the response.
    assert response_dict['success'] is True
    result = response_dict['result']
    #pprint.pprint(result)

    if (len(response_dict['result']) > 0):
        df = json_normalize(response_dict['result'])
    
        for item in df['resources']:
            # Check if there is any data and if the list has items
            if (item != None and len(item) > 0):
                dfj_worker = json_normalize(item)
                dfj = dfj.append(dfj_worker, ignore_index=True)
    
        print(dfj.shape)
        my_offset = my_offset + offset_step
    else:
        still_has_data = False

print('Done after offset ' + str(my_offset))

Offset: 0
(360, 27)
Offset: 250
(651, 27)
Offset: 500
(919, 27)
Offset: 750
(1196, 27)
Offset: 1000
(1529, 27)
Offset: 1250
(1870, 27)
Offset: 1500
(2237, 27)
Offset: 1750
(2601, 27)
Offset: 2000
(2971, 27)
Offset: 2250
(3379, 27)
Offset: 2500
(3761, 27)
Offset: 2750
(4092, 27)
Offset: 3000
(4479, 27)
Offset: 3250
(4870, 27)
Offset: 3500
(5285, 27)
Offset: 3750
(5674, 27)
Offset: 4000
(6016, 27)
Offset: 4250
(6397, 27)
Offset: 4500
(6802, 27)
Offset: 4750
(7216, 27)
Offset: 5000
(7553, 27)
Offset: 5250
(7940, 27)
Offset: 5500
(8360, 27)
Offset: 5750
(8755, 27)
Offset: 6000
(9138, 27)
Offset: 6250
(9461, 27)
Offset: 6500
(9893, 27)
Offset: 6750
(10280, 27)
Offset: 7000
(10668, 27)
Offset: 7250
(11027, 27)
Offset: 7500
(11397, 27)
Offset: 7750
(11769, 27)
Offset: 8000
(12109, 27)
Offset: 8250
(12480, 27)
Offset: 8500
(12805, 27)
Offset: 8750
(13221, 27)
Offset: 9000
(13598, 27)
Offset: 9250
(13954, 27)
Offset: 9500
(14375, 27)
Offset: 9750
(14834, 27)
Offset: 10000
(15234, 27)
Offset: 10

Hamburg:

Offset: 64500
(96938, 27)
Offset: 64750
Done after offset 64750

In [25]:
dfj.to_csv('transparenz-hamburg-ressources.csv')

In [ ]:
dfj.dtypes

In [134]:
dfj[['file_size','size']] = dfj[['file_size','size']].apply(pd.to_numeric)
dfj.dtypes

In [136]:
dfj['human_size'] = dfj['size'] / 1024/1024/1024
dfj[['size', 'format', 'name', 'human_size']].sort_values('size', ascending=False)

,size,format,name,human_size
5134,15972052378,ZIP,"Download DOP 20cm, März/April 2014 (14,8 GB, JPG)",1.487513e+01
5135,15840717308,ZIP,"Download DOP 20cm, April/Juni 2015 (14,7 GB, JPG)",1.475282e+01
8334,6242290610,ZIP,Download ALKIS Liegenschaftskarte - ausgewählt...,5.813586e+00
14490,6242290610,jpg,Download ALKIS Liegenschaftskarte - ausgewählt...,5.813586e+00
6103,6242290610,ZIP,Download ALKIS Liegenschaftskarte - ausgewählt...,5.813586e+00
8330,5635219857,ZIP,Download ALKIS Liegenschaftskarte - ausgewählt...,5.248207e+00
14503,5635219857,jpg,Download ALKIS Liegenschaftskarte - ausgewählt...,5.248207e+00
8341,5612032142,ZIP,Download ALKIS Liegenschaftskarte - ausgewählt...,5.226612e+00
14497,5612032142,jpg,Download ALKIS Liegenschaftskarte - ausgewählt...,5.226612e+00
6094,5612032142,ZIP,Download ALKIS Liegenschaftskarte - ausgewählt...,5.226612e+00


In [138]:
# Save to Excel
writer = pd.ExcelWriter(
    'Transparenz_Hamburg_Resources.xlsx', engine='xlsxwriter')
dfj.to_excel(writer, 'Tabelle1')
writer.save()

D:\Development\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:830: UserWarning: Ignoring URL 'http://geodienste.hamburg.de/HH_WFS_RISE_FG?service=WFS&request=GetFeature&version=1.1.0&TypeName=FD_FHH_Map_BSU_WSB2_RISE_FG:Soziale_Stadt,FD_FHH_Map_BSU_WSB2_RISE_FG:Aktive_Stadt_und_Ortsteilzentren,FD_FHH_Map_BSU_WSB2_RISE_FG:Hamburgische_Landesfoerderung,FD_FHH_Map_BSU_WSB2_RISE_FG:Staedtebauliche_Sanierung,FD_FHH_Map_BSU_WSB2_RISE_FG:Stadtumbau_West' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
D:\Development\Anaconda\lib\site-packages\xlsxwriter\worksheet.py:830: UserWarning: Ignoring URL 'http://geodienste.hamburg.de/HH_WFS_PflegeuGesundheit_BAE?service=WFS&request=GetFeature&VERSION=1.1.0&typename=hh_bae_gesundheit_aerzte,hh_bae_gesundheit_apotheken,hh_bae_gesundheit_seniorentreff,hh_bae_gesundheit_krankenkassen,hh_bae_gesundheit_krankenhaeuser,hh_bae_gesundheit_ambpfldienst,hh_bae_gesundheit_sportubewegung,hh_bae_gesun

This listing didn't include csv files. Might be connected to the limit (was 500 initially), but the job now already takes a while. 

<h2>Second approach</h2>

So why not search?

My results (see checks below): Search makes it hard to search with OR (which it can't) or to sort DESC (which it can't) so still neccessary to load all ressources like up there (takes an hour or two)

In [3]:
# Use the json module to dump a dictionary to a string for posting.
data_string = urllib.parse.quote(json.dumps({'id': 'data-explorer', 'query': 'format:csv'}))#, 'order_by': 'size', 'limit': 50}))

#py 3 http://stackoverflow.com/questions/13175253/python-error-type-error-post-data-should-be-bytes-also-user-agent-issue
data_string = data_string.encode('ascii')


# Make the HTTP request.
#response = urllib2.urlopen('http://suche.transparenz.hamburg.de/api/action/group_list',
#        data_string)
# py 3
response = urlopen('http://suche.transparenz.hamburg.de/api/action/resource_search', data_string)
#response = urlopen('https://www.europeandataportal.eu/data/en/api/action/current_package_list_with_resources', data_string)
#response = urlopen('https://data.gov.uk/api/action/resource_search', data_string)

assert response.code == 200

# Use the json module to load CKAN's response into a dictionary.
response_dict = json.loads(response.read())

# Check the contents of the response.
assert response_dict['success'] is True
result = response_dict['result']

dfs = json_normalize(response_dict['result'])

dfs

,count,results
0,1932,[{'resource_group_id': '000e0380-6636-4986-b87...


In [4]:
dfs.dtypes

dflist = DataFrame()

for listentry in dfs['results']:
    dflist_worker = json_normalize(listentry)
    dflist = dflist.append(dflist_worker)

dflist

,cache_last_updated,cache_url,created,description,file_size,format,hash,id,language,last_modified,...,resource_type,revision_id,size,state,tracking_summary.recent,tracking_summary.total,url,verified,webstore_last_updated,webstore_url
0,None,None,2017-01-27T14:22:29.205568,,10330,CSV,f654ae7e6c16ac5f3fe29d715abbb2c8a1b64cfeec976a...,a3053047-fef1-4251-a6ce-546925e977be,,None,...,None,32c46dd2-1ef0-4f9f-9b33-a0aaee8f549f,10330,active,0,0,http://daten.transparenz.hamburg.de/Dataport.H...,NaN,None,None
1,None,None,2017-01-27T11:51:31.442849,,8100,CSV,2c0f7bbd5e495b25b1adb204a23a614c6d0cb6fdbc3d40...,0e738106-ab79-48a1-a2a5-dc7abb46c753,,None,...,None,eaeb7140-6120-442b-af90-88a2efc152ea,8100,active,0,0,http://daten.transparenz.hamburg.de/Dataport.H...,NaN,None,None
2,None,None,2017-01-27T22:54:34.192478,,9003,CSV,9003,d1f6e055-ec4c-41d6-89da-84427b5a5977,,None,...,None,dbeebb34-b23d-4146-92b8-a1f36be5e375,9003,active,0,0,http://daten.transparenz.hamburg.de/Dataport.H...,NaN,None,None
3,None,None,2017-01-27T13:32:39.168997,,11222,CSV,4ecf130c0b171331a152af6b74fad55037b34fe5f6d278...,e7244696-ffae-49df-b8e9-75f77dfb77bc,,None,...,None,36914024-c506-48e1-a53a-c64beaf3a335,11222,active,0,0,http://daten.transparenz.hamburg.de/Dataport.H...,NaN,None,None
4,None,None,2017-01-27T13:05:33.872708,,9154,CSV,23f7e703781f8de22ae070d7e8d19d9f5b2f0a4097c978...,9bb23a1f-7eb8-4da4-a3ed-54e177093f37,,None,...,None,c7e015d1-e5f5-4160-bc28-b29d0c72f324,9154,active,0,0,http://daten.transparenz.hamburg.de/Dataport.H...,NaN,None,None
5,None,None,2017-01-27T16:54:16.871296,,8902,CSV,7fed83154f230942a81c678e5115aabfe03be4fb2dc397...,463e04b9-affc-42dd-a6b5-489dd22c21c1,,None,...,None,5a54a53e-e34b-4e66-883f-dfafae811dd6,8902,active,0,0,http://daten.transparenz.hamburg.de/Dataport.H...,NaN,None,None
6,None,None,2017-01-27T16:54:24.122427,,8733,CSV,95434699b332c5688a64b369982b6a9b73cd11162c165c...,74b57421-8ab4-47f0-83bc-6fcc978d535a,de,None,...,None,e560eaee-61b2-4703-ae33-d18ebc5bf3b2,8733,active,0,0,http://daten.transparenz.hamburg.de/Dataport.H...,NaN,None,None
7,None,None,2017-01-27T12:36:00.389671,,10069,CSV,45c60190ce6c3eeb153533987e8ef29d85832527f1cf66...,0edfad53-a33d-4983-9ae1-ed780d6ca6ae,,None,...,None,aebbbd33-ad0c-4d56-81e0-e4fe0b72cbef,10069,active,0,0,http://daten.transparenz.hamburg.de/Dataport.H...,NaN,None,None
8,None,None,2017-04-13T20:13:04.036526,,638,CSV,674596fc02bb6bd0a613fec76bc383d649a320c17c9b4e...,63acb6b2-53ff-41e7-a7ca-facf5bad7e47,de,None,...,None,8bb073a6-06a2-4516-a3c9-1cfdad0ab592,638,active,0,0,http://daten.transparenz.hamburg.de/Dataport.H...,NaN,None,None
9,None,None,2017-01-27T13:48:50.992020,,9486,CSV,eefaa56deb26e13a02e3452361ddaf9be5a14923d2b003...,eee43d59-3915-4cf1-9d5a-12926c9fadf6,de,None,...,None,c27a55a6-b547-4d68-bee0-58a81a606c3a,9486,active,0,0,http://daten.transparenz.hamburg.de/Dataport.H...,NaN,None,None


In [5]:
dflist.dtypes

cache_last_updated         object
cache_url                  object
created                    object
description                object
file_size                  object
format                     object
hash                       object
id                         object
language                   object
last_modified              object
mimetype                   object
mimetype_inner             object
name                       object
position                    int64
resource_group_id          object
resource_type              object
revision_id                object
size                       object
state                      object
tracking_summary.recent     int64
tracking_summary.total      int64
url                        object
verified                   object
webstore_last_updated      object
webstore_url               object
dtype: object

In [6]:
#dflist[['file_size','size']] = dflist[['file_size','size']].apply(pd.to_numeric)
dflist[['size']] = dflist[['size']].apply(pd.to_numeric)
dflist.dtypes

cache_last_updated         object
cache_url                  object
created                    object
description                object
file_size                  object
format                     object
hash                       object
id                         object
language                   object
last_modified              object
mimetype                   object
mimetype_inner             object
name                       object
position                    int64
resource_group_id          object
resource_type              object
revision_id                object
size                        int64
state                      object
tracking_summary.recent     int64
tracking_summary.total      int64
url                        object
verified                   object
webstore_last_updated      object
webstore_url               object
dtype: object

In [7]:
dflists = dflist.sort_values('size',ascending=False)
dflists[['name', 'format', 'url', 'size']]



,name,format,url,size
1872,"Download Bodenrichtwerte, 2010 (CSV)",CSV,http://archiv.transparenz.hamburg.de/hmbtgarch...,507275545
1892,"Download Bodenrichtwerte, 2010 (CSV)",CSV,http://Daten-hamburg.de/infrastruktur_bauen_wo...,507275545
1924,"Download Bodenrichtwerte, 2010 (CSV)",CSV,http://Daten-hamburg.de/infrastruktur_bauen_wo...,507275545
1877,"Download Bodenrichtwerte, 2012 (CSV)",CSV,http://archiv.transparenz.hamburg.de/hmbtgarch...,507266020
1896,"Download Bodenrichtwerte, 2012 (CSV)",CSV,http://Daten-hamburg.de/infrastruktur_bauen_wo...,507266020
1926,"Download Bodenrichtwerte, 2012 (CSV)",CSV,http://Daten-hamburg.de/infrastruktur_bauen_wo...,507266020
1875,"Download Bodenrichtwerte, 2011 (CSV)",CSV,http://archiv.transparenz.hamburg.de/hmbtgarch...,507264519
1894,"Download Bodenrichtwerte, 2011 (CSV)",CSV,http://Daten-hamburg.de/infrastruktur_bauen_wo...,507264519
1925,"Download Bodenrichtwerte, 2011 (CSV)",CSV,http://Daten-hamburg.de/infrastruktur_bauen_wo...,507264519
1916,"Download Bodenrichtwerte, 2014 (CSV)",CSV,http://Daten-hamburg.de/infrastruktur_bauen_wo...,461187706


In [8]:
for itera in dflists['url'].head(5):
    print(itera)

http://archiv.transparenz.hamburg.de/hmbtgarchive/HMDK/br0200001-hamburg2010_2110_snap_1_16158_snap_1.CSV
http://Daten-hamburg.de/infrastruktur_bauen_wohnen/bodenrichtwerte/br0200001-Hamburg2010.csv
http://Daten-hamburg.de/infrastruktur_bauen_wohnen/bodenrichtwerte/br0200001-Hamburg2010.csv
http://archiv.transparenz.hamburg.de/hmbtgarchive/HMDK/br0200001-hamburg2012_2112_snap_1_16160_snap_1.CSV
http://Daten-hamburg.de/infrastruktur_bauen_wohnen/bodenrichtwerte/br0200001-Hamburg2012.csv
